# Book Recommendation

![Network diagram](../outputs/edge_list_gephi.png)

In [1]:
import pandas as pd
import networkx as nx

from typing import Any

from ipywidgets import interact, Select
from IPython.display import display
        

In [2]:
# Load and clean data
read_ticks_df = pd.read_csv("../datasets/dataset.csv")
read_ticks_df["date_created"] = pd.to_datetime(read_ticks_df["date_created"].str.strip().str.replace('"', ''))

In [3]:
# For each user, sort the ticks based on the date created. 

EDGE_COLUMNS = ["source", "target"]

def create_edge_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values("date_created")
    if len(df) <= 1:
        # Return empty dataframe
        return pd.DataFrame([], columns=EDGE_COLUMNS)
    return pd.DataFrame(
        {
            "source": df["book"].iloc[:-1].values,
            "target": df["book"].iloc[1:].values,
        }
    )


edge_list_df = read_ticks_df.groupby("user_id").apply(create_edge_df).reset_index(0).reset_index(drop=True)

In [4]:
# Create Directed Graph
graph = nx.from_pandas_edgelist(edge_list_df, create_using=nx.DiGraph)

In [5]:
# Create WebApp

class GraphRecommendationApp:
    """
    Deploys a Recommendation App
    * Takes a directed graph as input. 
    * Computes centrality.
    * Deploys an app to recommend best node based on centrality.
    """

    def __init__(self, graph: nx.DiGraph) -> None:
        self.graph = graph.copy()
        self.centrality_df = self._compute_centrality()

    def _compute_centrality(self) -> pd.DataFrame:
        # Compute Centrality
        out=nx.out_degree_centrality(self.graph)
        nx.set_node_attributes(self.graph, out, 'out-degree')
        bb = nx.betweenness_centrality(self.graph)
        nx.set_node_attributes(self.graph, bb, 'betweenness')
        eigen= nx.eigenvector_centrality(self.graph)
        nx.set_node_attributes(self.graph, eigen, 'eigen')

        return pd.DataFrame(
            [
                {"node": node_name, **node_data}
                for node_name, node_data in self.graph.nodes(data=True)
            ]
        )
    
    def get_nodes(self) -> list[Any]:
        return self.centrality_df["node"].sort_values().unique().tolist()
    
    def view(self, book=''):
        row1=[n for n in graph[book]]
        df2= pd.DataFrame(row1)
        df3=df2.set_index(0).join(self.centrality_df.set_index('node'))
        df3=df3.sort_values(by=['eigen'],ascending=False)
        return df3
    
    def display_app(self) -> None:
        w = Select(options=self.get_nodes())
        v = interact(self.view, book=w)
        display(v)
        

In [6]:
book_rec = GraphRecommendationApp(graph)
book_rec.display_app()

interactive(children=(Select(description='book', options=('abernethy2', 'abraham', 'adzic', 'alag', 'allaire',…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>